1. Write data to delta lake (managed table)
2. Write data to delta lake (external table)
3. Read data from delta lake (Table)
4. Read data from delta lake (File)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS f1_demo
LOCATION '/mnt/formula1dlks7/demo'

In [0]:
results_df = spark.read \
  .option("inferSchema", True) \
  .json("/mnt/formula1dlks7/raw/2021-03-28/results.json")

In [0]:
results_df.write.format("delta").mode("overwrite").saveAsTable("f1_demo.results_managed")

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

In [0]:
results_df.write.format("delta").mode("overwrite").save("/mnt/formula1dlks7/demo/results_external")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.results_external
USING DELTA
LOCATION '/mnt/formula1dlks7/demo/results_external'

In [0]:
%sql
SELECT * FROM f1_demo.results_external

In [0]:
results_external_df = spark.read.format("delta").load("/mnt/formula1dlks7/demo/results_external")

In [0]:
display(results_external_df)

In [0]:
results_df.write.format("delta").mode("overwrite").partitionBy("constructorId").saveAsTable("f1_demo.results_partitioned")

In [0]:
%sql
SHOW PARTITIONS f1_demo.results_partitioned

1. Update Delta Table
2. Delete From Delta Table

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

In [0]:
%sql
UPDATE f1_demo.results_managed
  SET points = 11 - position
  WHERE position <= 10

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, '/mnt/formula1dlks7/demo/results_managed')
deltaTable.update("position <= 10", {"points": '21 - position'})

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

In [0]:
%sql
DELETE FROM f1_demo.results_managed
WHERE position > 10;

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, '/mnt/formula1dlks7/demo/results_managed')
deltaTable.delete("points = 0")

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

Upsert using merge

In [0]:
drivers_day1_df = spark.read \
  .option("inferSchema", True) \
  .json("/mnt/formula1dlks7/raw/2021-03-28/drivers.json") \
  .filter("driverId <= 10") \
  .select("driverId", "dob", "name.forename", "name.surname")

In [0]:
display(drivers_day1_df)

In [0]:
drivers_day1_df.createOrReplaceTempView("drivers_day1")

In [0]:
from pyspark.sql.functions import upper

drivers_day2_df = spark.read \
  .option("inferSchema", True) \
  .json("/mnt/formula1dlks7/raw/2021-03-28/drivers.json") \
  .filter("driverId BETWEEN 6 AND 15") \
  .select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))

In [0]:
display(drivers_day2_df)

In [0]:
drivers_day2_df.createOrReplaceTempView("drivers_day2")

In [0]:
from pyspark.sql.functions import upper

drivers_day3_df = spark.read \
  .option("inferSchema", True) \
  .json("/mnt/formula1dlks7/raw/2021-03-28/drivers.json") \
  .filter("driverId BETWEEN 1 AND 5 OR driverId BETWEEN 16 AND 20") \
  .select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_merge (
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING DELTA;

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day1 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN 
  UPDATE SET tgt.dob = upd.dob,
              tgt.forename = upd.forename,
              tgt.surname = upd.surname,
              tgt.updatedDate = current_timestamp
WHEN NOT MATCHED 
  THEN INSERT (driverId, dob, forename, surname, createdDate) VALUES (driverId, dob, forename, surname, current_timestamp)

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day2 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN 
  UPDATE SET tgt.dob = upd.dob,
              tgt.forename = upd.forename,
              tgt.surname = upd.surname,
              tgt.updatedDate = current_timestamp
WHEN NOT MATCHED 
  THEN INSERT (driverId, dob, forename, surname, createdDate) VALUES (driverId, dob, forename, surname, current_timestamp)

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_timestamp

deltaTable = DeltaTable.forPath(spark, '/mnt/formula1dlks7/demo/drivers_merge')
deltaTable.alias("tgt").merge(
    drivers_day3_df.alias("upd"),
     "tgt.driverId = upd.driverId") \
    .whenMatchedUpdate(set = {"dob": "upd.dob", 
                              "forename": "upd.forename", 
                              "surname": "upd.surname",
                              "updatedDate": "current_timestamp()"}) \
    .whenNotMatchedInsert(
        values = {"driverId": "upd.driverId", 
                  "dob": "upd.dob", 
                  "forename": "upd.forename", 
                  "surname": "upd.surname",
                  "createdDate": "current_timestamp()"}
    ) \
    .execute()

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

1. History & versioning
2. Time Travel
3. Vaccum

In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge VERSION AS OF 2;

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2025-04-08T13:15:30.000+00:00';

In [0]:
df = spark.read.format("delta").option("timestampAsOf", '2025-04-08T13:15:30.000+00:00').load("/mnt/formula1dlks7/demo/drivers_merge")

In [0]:
display(df)

In [0]:
df = spark.read.format("delta").option("versionAsOf", '2').load("/mnt/formula1dlks7/demo/drivers_merge")

In [0]:
display(df)

In [0]:
%sql
VACUUM f1_demo.drivers_merge;

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2025-04-08T13:15:30.000+00:00';

In [0]:
# %sql
# SET spar.k.databricks.delta.retentionDurationCheck.enabled = false;
# VACUUM f1_demo.drivers_merge RETAIN 0 HOURS;

In [0]:
# %sql
# SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2025-04-08T13:15:30.000+0000';

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge

In [0]:
%sql
DELETE FROM f1_demo.drivers_merge WHERE driverId = 1;

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING f1_demo.drivers_merge VERSION AS OF 5 src
ON tgt.driverId = src.driverId
WHEN NOT MATCHED THEN INSERT *

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

Transaction Logs

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_txn (
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING DELTA;

In [0]:
%sql
DESC HISTORY f1_demo.drivers_txn;

In [0]:
%sql
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge
WHERE driverId = 1;

In [0]:
%sql
-- CONVERT TO DELTA f1_demo.<table_name>

In [0]:
%sql
-- CONVERT TO DELTA parquet.`<path>(/mnt/formula1dlks7/demo/<table_name>)`